In [ ]:
import pandas as pd
d=pd.read_csv('/content/Restaurant_Reviews.csv')
df=pd.DataFrame(d)
df

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [ ]:
df.isnull().sum()

Review    0
Liked     0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Liked   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


Data preprocessing

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')
def preprocessing(text):
  text=text.lower()
  review_words=text.split()
  review=[words for words in review_words if not words in set(stopwords.words('english'))]
  ps=PorterStemmer()
  review=[ps.stem(word) for word in review]
  review=[lemmatizer.lemmatize(word) for word in review]
  return " ".join(review)
preprocessing('Wow... Loved this place.')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'wow... love place.'

In [ ]:
import spacy
nlp=spacy.load('en_core_web_sm')
def preprocess(text):
  doc=nlp(text)
  no_stop_word=[]
  for token in doc:
    if not token.is_stop:
      no_stop_word.append(token.text)
  return " ".join(no_stop_word)
preprocess('Wow... Loved this place.')



'Wow ... Loved place .'

In [ ]:
df['filtered_review']=df.Review.apply(lambda x: preprocessing(x))

In [ ]:
df['filtered_review']

0                                     wow... love place.
1                                            crust good.
2                                    tasti textur nasty.
3      stop late may bank holiday rick steve recommen...
4                              select menu great prices.
                             ...                        
995                    think food flavor textur lacking.
996                              appetit instantli gone.
997                        overal impress would go back.
998    whole experi underwhelming, think we'll go nin...
999    then, wast enough life there, pour salt wound ...
Name: filtered_review, Length: 1000, dtype: object

In [ ]:
df.head(20)

,Review,Liked,filtered_review
0,Wow... Loved this place.,1,wow... love place.
1,Crust is not good.,0,crust good.
2,Not tasty and the texture was just nasty.,0,tasti textur nasty.
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great prices.
5,Now I am getting angry and I want my damn pho.,0,get angri want damn pho.
6,Honeslty it didn't taste THAT fresh.),0,honeslti tast fresh.)
7,The potatoes were like rubber and you could te...,0,potato like rubber could tell made ahead time ...
8,The fries were great too.,1,fri great too.
9,A great touch.,1,great touch.


Sentiment analysis

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time


In [ ]:
t=time.time()
nltk.download('vader_lexicon')
analyzer=SentimentIntensityAnalyzer()
nlp_sentiment_score=[]
for word in df['Review']:
  score=analyzer.polarity_scores(word)
  nlp_sentiment_score.append(score)
print(f'semtiment scoring time :{time.time()-t:.2f} seconds')

semtiment scoring time :0.21 seconds


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
df['positive_sentiment']=[score['pos'] for score in nlp_sentiment_score]
df['negative_sentiment']=[score['neg'] for score in nlp_sentiment_score]
df['neutral_sentiment']=[score['neu'] for score in nlp_sentiment_score]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scalar=MinMaxScaler()


In [ ]:
import numpy as np
df['positive_sentiment']=min_max_scalar.fit_transform(np.array(df['positive_sentiment']).reshape(-1,1))
df['negative_sentiment']=min_max_scalar.fit_transform(np.array(df['negative_sentiment']).reshape(-1,1))
df['neutral_sentiment']=min_max_scalar.fit_transform(np.array(df['neutral_sentiment']).reshape(-1,1))

In [ ]:
df.head()

,Review,Liked,filtered_review,positive_sentiment,negative_sentiment,neutral_sentiment
0,Wow... Loved this place.,1,wow... love place.,0.565,0.000,0.435
1,Crust is not good.,0,crust good.,0.000,0.445,0.555
2,Not tasty and the texture was just nasty.,0,tasti textur nasty.,0.000,0.340,0.660
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...,0.322,0.093,0.585
4,The selection on the menu was great and so wer...,1,select menu great prices.,0.272,0.000,0.728


count vectorization

In [ ]:
df['word_count']=df['Review'].apply(lambda x: (x.count(" ")+1)/50)

In [ ]:
df.head()

,Review,Liked,filtered_review,positive_sentiment,negative_sentiment,neutral_sentiment,word_count
0,Wow... Loved this place.,1,wow... love place.,0.565,0.000,0.435,0.08
1,Crust is not good.,0,crust good.,0.000,0.445,0.555,0.08
2,Not tasty and the texture was just nasty.,0,tasti textur nasty.,0.000,0.340,0.660,0.16
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...,0.322,0.093,0.585,0.30
4,The selection on the menu was great and so wer...,1,select menu great prices.,0.272,0.000,0.728,0.24


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer=CountVectorizer()
x=vectorizer.fit_transform(df['filtered_review']).toarray()
y=df.iloc[:,1].values

In [ ]:
df.iloc[:,1]

0      1
1      0
2      0
3      1
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: Liked, Length: 1000, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
x_train.shape

(800, 1853)

In [ ]:
x_test.shape

(200, 1853)

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

RandomForest

In [ ]:
model1=RandomForestClassifier()
model1.fit(x_train,y_train)
y_pred=model1.predict(x_test)

In [ ]:
y_pred

array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1])

In [ ]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.82      0.66      0.73       120
           1       0.61      0.79      0.68        80

    accuracy                           0.71       200
   macro avg       0.71      0.72      0.71       200
weighted avg       0.74      0.71      0.71       200



In [ ]:
model1=GaussianNB()
model1.fit(x_train,y_train)
y_pred=model1.predict(x_test)

In [ ]:
y_pred

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1])

In [ ]:
print(accuracy_score(y_pred,y_test))

0.685


In [ ]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.49      0.77      0.60        61
           1       0.87      0.65      0.74       139

    accuracy                           0.69       200
   macro avg       0.68      0.71      0.67       200
weighted avg       0.75      0.69      0.70       200



Multinomial Naive Bayes

In [ ]:
model2=MultinomialNB()
model2.fit(x_train,y_train)
y_pred2=model2.predict(x_test)
print(accuracy_score(y_pred,y_test))

0.685


In [ ]:
x_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Hyperparameter tuning

In [ ]:
vectorizer.get_feature_names_out()[1200]

'perfect'

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
cross_val_score(SVC(gamma='auto',C=30,kernel='linear'),x,y,cv=5)

array([0.74 , 0.7  , 0.695, 0.74 , 0.7  ])

In [ ]:
classifier=GridSearchCV(SVC(gamma='auto'),{
                            'C':[10,20,30],'kernel':['rbf','linear'],
},cv=5,return_train_score=False)

In [ ]:
classifier.fit(x,y)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [10, 20, 30], 'kernel': ['rbf', 'linear']})

In [ ]:
pd.DataFrame(classifier.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.584398,0.058487,0.358151,0.036414,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.620,0.570,0.570,0.645,0.645,0.610,0.033912,6
1,0.438073,0.035983,0.071747,0.003232,10,linear,"{'C': 10, 'kernel': 'linear'}",0.750,0.705,0.715,0.790,0.730,0.738,0.030100,1
2,0.477935,0.007681,0.267725,0.014201,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.650,0.655,0.645,0.705,0.700,0.671,0.025962,5
3,0.420137,0.013268,0.069831,0.002845,20,linear,"{'C': 20, 'kernel': 'linear'}",0.745,0.700,0.700,0.750,0.715,0.722,0.021587,2
4,1.080003,0.452271,0.492211,0.091730,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.685,0.695,0.670,0.730,0.730,0.702,0.024207,4
5,0.953203,0.180705,0.110373,0.049327,30,linear,"{'C': 30, 'kernel': 'linear'}",0.740,0.700,0.695,0.740,0.700,0.715,0.020494,3


In [ ]:
model_params={
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
        'kernel':['linear','rbf'],
        'C':[1,10,20],
    }
    },
    'random_forest':{
        'model':
         RandomForestClassifier(),
    'params':{
        'n_estimators':[1,5,10]
    }
    },
    'logistic_regression':
    {
        'model':LogisticRegression(multi_class='auto',solver='liblinear'),
        'params':{
            'C':[1,5,10]
        }
    },

}

In [ ]:
scores=[]
for model_name,model_param in model_params.items():
  clf2=GridSearchCV(model_param['model'],model_param['params'],return_train_score=False)
  clf2.fit(x,y)
  scores.append({
      'model': model_name,
      'best_score': clf2.best_score_,
      'best params': clf2.best_params_
  })

In [ ]:
pd.DataFrame(scores)

,model,best_score,best params
0,svm,0.770,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.711,{'n_estimators': 5}
2,logistic_regression,0.762,{'C': 10}


In [ ]:
x_train_2d=np.stack(x_train)
x_test_2d=np.stack(x_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scale_train_embed=scaler.fit_transform(x_train_2d)
scale_test_embed=scaler.fit_transform(x_test_2d)

In [ ]:
x_train_embed=np.array(scale_train_embed).reshape(-1,1)
x_test_embed=np.array(scale_test_embed).reshape(-1,1)

MultinomialNB after HyperParameterTuning

In [ ]:
clf=MultinomialNB()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
print("accuracy score is: {}".format(accuracy_score(y_pred,y_test)))

print(classification_report(y_pred,y_test))

accuracy score is: 0.75
              precision    recall  f1-score   support

           0       0.74      0.74      0.74        96
           1       0.76      0.76      0.76       104

    accuracy                           0.75       200
   macro avg       0.75      0.75      0.75       200
weighted avg       0.75      0.75      0.75       200



In [ ]:
clf2=RandomForestClassifier()
clf2.fit(x_train,y_train)
y_pred=clf2.predict(x_test)
print("accuracy score is: {}".format(accuracy_score(y_pred,y_test)))

accuracy score is: 0.685


Logistic Regression

In [ ]:
clf2=LogisticRegression()
clf2.fit(x_train_2d,y_train)
y_pred=clf2.predict(x_test_2d)
print("accuracy score is: {}".format(accuracy_score(y_pred,y_test)))
print(classification_report(y_pred,y_test))

accuracy score is: 0.735
              precision    recall  f1-score   support

           0       0.80      0.69      0.74       111
           1       0.67      0.79      0.73        89

    accuracy                           0.73       200
   macro avg       0.74      0.74      0.73       200
weighted avg       0.74      0.73      0.74       200



SVM

In [ ]:
clf3=SVC(kernel='linear',C=20,gamma='auto')
clf3.fit(x_train,y_train)
y_pred=clf3.predict(x_test)
print("accuracy score is: {}".format(accuracy_score(y_pred,y_test)))
print(classification_report(y_pred,y_test))

accuracy score is: 0.7
              precision    recall  f1-score   support

           0       0.75      0.67      0.71       108
           1       0.65      0.74      0.69        92

    accuracy                           0.70       200
   macro avg       0.70      0.70      0.70       200
weighted avg       0.71      0.70      0.70       200



Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf4=DecisionTreeClassifier()
clf4.fit(x_train,y_train)
y_pred=clf4.predict(x_test)
print("accuracy score is: {}".format(accuracy_score(y_pred,y_test)))
print(classification_report(y_pred,y_test))

accuracy score is: 0.71
              precision    recall  f1-score   support

           0       0.82      0.66      0.73       120
           1       0.61      0.79      0.68        80

    accuracy                           0.71       200
   macro avg       0.71      0.72      0.71       200
weighted avg       0.74      0.71      0.71       200



In [ ]:
def prediction(text):
  text=text.lower()
  review_words=nlp(text)
  review=[words for words in review_words if not words in set(stopwords.words('english'))]
  review=" ".join(str(v) for v in review)
  ps=PorterStemmer()
  review=[ps.stem(word) for word in review]
  review=[lemmatizer.lemmatize(word) for word in review]
  review=" ".join(review)
  #return review
  final_review=vectorizer.transform([review]).toarray()
  #review=np.array(review).reshape(-1,1)
  return model1.predict(final_review)
prediction('lovely food here.')


array([1])

In [ ]:
text='great food here'
if prediction(text):
  print('It is positive review')
else:
  print('It is negative review')

It is positive review
